In [7]:
import os
from datetime import datetime
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon
from pathlib import Path
from loguru import logger
from IPython.display import clear_output

from multiplex_pipeline.utils.utils import load_analysis_settings
from multiplex_pipeline.core_preparation.channel_scanner import discover_channels, build_transfer_map
from multiplex_pipeline.core_preparation.controller import CorePreparationController
from multiplex_pipeline.core_preparation.file_io import GlobusFileStrategy, LocalFileStrategy

### Load analysis settings

In [8]:
# load analysis configuration
settings_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\analysis_settings_BLCA1B.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-1B/BLCA-1B_Final',
 'analysis_dir': 'R:/Wayne/BLCA/BLCA-1B_Analysis',
 'log_dir': WindowsPath('R:/Wayne/BLCA/BLCA-1B_Analysis/logs'),
 'detection_image': 'BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('R:/Wayne/BLCA/BLCA-1B_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('R:/Wayne/BLCA/BLCA-1B_Analysis/temp'),
 'cores_dir_output': WindowsPath('R:/Wayne/BLCA/BLCA-1B_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': 'instaseg',
  'model': 'fluorescence_nuclei_and_cells',
  'kwargs': {'pixel_

### Define the logger

In [9]:
log_file = settings['log_dir'] / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

4

### Define cores for processing

In [4]:
df_path = settings['core_info_file_path'].with_suffix('.pkl')

df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,640.0,4992.0,3136.0,8832.0,rectangle,"[[4992.0, 3136.0], [640.0, 3136.0], [640.0, 88..."
1,Core_001,960.0,5632.0,9920.0,15680.0,rectangle,"[[5632.0, 9920.0], [960.0, 9920.0], [960.0, 15..."
2,Core_002,1664.0,5504.0,16896.0,22656.0,rectangle,"[[5504.0, 16896.0], [1664.0, 16896.0], [1664.0..."
3,Core_003,1728.0,5952.0,23680.0,29312.0,rectangle,"[[5952.0, 23680.0], [1728.0, 23680.0], [1728.0..."
4,Core_004,1856.0,6400.0,30400.0,35712.0,rectangle,"[[6400.0, 30400.0], [1856.0, 30400.0], [1856.0..."


In [5]:
# create a subset of cores (optional)
df = df[5:7]
df

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
5,Core_005,6144.0,11904.0,2752.0,8576.0,rectangle,"[[11904.0, 2752.0], [6144.0, 2752.0], [6144.0,..."
6,Core_006,6720.0,12096.0,9664.0,15168.0,rectangle,"[[12096.0, 9664.0], [6720.0, 9664.0], [6720.0,..."


## Local files

In [5]:
channel_map = discover_channels(Path(settings['image_dir']), 
                                include_channels=settings['include_channels'], 
                                exclude_channels=settings['exclude_channels'], 
                                use_markers=settings.get('use_markers'), 
                                ignore_markers=settings.get('ignore_markers'))

2025-09-22 21:15:46.633 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:85 - Discovered 131 channels:
2025-09-22 21:15:46.634 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:87 - 001_CDC25C <- R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_FITC_CDC25C-AF488_FINAL_AFR_F.ome.tif
2025-09-22 21:15:46.635 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:87 - 001_DAPI <- R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-09-22 21:15:46.635 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:87 - 001_cycD3 <- R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_Cy7_cycD3-AF750_FINAL_AFR_F.ome.tif
2025-09-22 21:15:46.635 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:87 - 001_pH2AX <- R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR

In [ ]:
# for tests you can request a small set of channels here
# then send 'short_map' to the controller instead of 'channel_map'
selected_keys = ["DAPI", "CD3"]
short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
short_map

{'DAPI': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [ ]:
strategy = LocalFileStrategy()

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = settings['cores_dir_tif'],
    output_dir = settings['cores_dir_output'],
    file_strategy = strategy,
    margin = settings['core_cutting']['margin'],
    mask_value = settings['core_cutting']['mask_value'],
    max_pyramid_levels = settings['core_cutting']['max_pyramid_level'],
    chunk_size = settings['core_cutting']['chunk_size'],
)

controller.run()

2025-09-22 21:15:56.122 | INFO     | multiplex_pipeline.core_preparation.controller:run:111 - Starting controller run loop...
2025-09-22 21:16:01.377 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:87 - Cut and saved core Core_000, channel DAPI
2025-09-22 21:16:06.291 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:87 - Cut and saved core Core_001, channel DAPI
2025-09-22 21:16:09.525 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:87 - Cut and saved core Core_002, channel DAPI
2025-09-22 21:16:11.863 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:87 - Cut and saved core Core_003, channel DAPI
2025-09-22 21:16:14.280 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:87 - Cut and saved core Core_004, channel DAPI
2025-09-22 21:16:19.498 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:87 - Cut and saved core Core_005, channel DAPI
2025-09-22 21:16